In [1]:
#!git clone https://github.com/ziatdinovmax/im2spec.git
%cd im2spec

/Users/adivatsavai/Adi/Curiosity RL/im2spec


In [2]:
import gym
from gym import spaces
import numpy as np
import pygame

import scipy
import gdown
import h5py
import matplotlib.pyplot as plt

import os
from copy import deepcopy as dc

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from im2spec.models import im2spec, spec2im
from im2spec.utils import create_training_set, predict, loop_area, encode, decode, latent_gmm
from im2spec.train_utils import trainer

import random
from collections import namedtuple, deque
import math

pygame 2.5.2 (SDL 2.28.3, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
#!gdown https://drive.google.com/uc?id=1yrGXukGspctsSYrHn3grY_isQR2VFPLd

In [4]:
h5 = h5py.File(r'BEPS_kyleSample_3um_0018.h5', 'r+')

spec_val = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_001/Spectroscopic_Values']


amp = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Amplitude [V]'].reshape(100,100,384)
pha = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Phase [rad]'].reshape(100,100,384)
fre = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Frequency [Hz]'].reshape(100,100,384)
q = h5['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Guess']['Quality Factor'].reshape(100,100,384)
#r2 = h5['Measurement_000/Channel_001/Raw_Data-SHO_Fit_000/Guess']['R2 Criterion'].reshape(100,100,384)
pola = (amp*np.cos(pha+2))*1E5


In [5]:
class environment(gym.Env):
    def __init__(self, image, spectra, start = [50, 50], image_patch = 5):
        super(environment, self).__init__()
        self.image_patch = image_patch
        self.radius = int((image_patch - 1)/2)+1
        self.image = image
        self.spectra = spectra
        self.color = 255/(image.max() - image.min()) * image - 255/(image.max() - image.min()) * image.min()
        self.num_rows = 100
        self.num_columns = 100
        self.pos_X = [start]
        self.X = []
        self.y = []
        self.X.append(self.image[start[0]-int((self.image_patch-1)/2): start[0] + int((self.image_patch+1)/2), start[1]-int((self.image_patch-1)/2): start[1] + int((self.image_patch+1)/2)])
        self.y.append(spectra[start[0], start[1]])
        self.seen = np.zeros([100, 100])
        self.pos = start
        self.pos_X = [start]
        
        for i in range(self.radius):
                for j in range(self.radius):
                    self.seen[start[0]+i, start[1]+j] = 1
                    self.seen[start[0]-i, start[1]+j] = 1
                    self.seen[start[0]+i, start[1]-j] = 1
                    self.seen[start[0]-i, start[1]-j] = 1
        
        
        self.observation_space = spaces.Tuple((spaces.Discrete(self.num_rows), spaces.Discrete(self.num_columns)))
        
        pygame.init()
        self.cell_size = 8
        
        self.screen = pygame.display.set_mode((self.num_columns * self.cell_size, self.num_rows * self.cell_size))
        
    
    def step(self, action):
        self.pos = action
        if not [action[0], action[1]] in self.pos_X:
            self.pos_X.append(action)
            for i in range(self.radius):
                for j in range(self.radius):
                    self.seen[action[0]+i, action[1]+j] = 1
                    self.seen[action[0]-i, action[1]+j] = 1
                    self.seen[action[0]+i, action[1]-j] = 1
                    self.seen[action[0]-i, action[1]-j] = 1
            
            ind = action
            self.X.append(self.image[ind[0]-int((self.image_patch-1)/2): ind[0] + int((self.image_patch+1)/2), ind[1]-int((self.image_patch-1)/2): ind[1] + int((self.image_patch+1)/2)])
            self.y.append(spectra[ind[0], ind[1]])
                    
    
    def state(self, model):
        pred_loop = predict(model, self.X[-1].reshape([1, 1, self.image_patch, self.image_patch])).reshape(64)
        
        reward = np.sum((pred_loop - self.y[-1])**2)
        state = [self.pos[0], self.pos[1]]
        state += list(self.X[-1].reshape(self.image_patch**2))
        state += list(self.y[-1])
        return(self.pos_X, np.array(self.X).reshape([len(self.X), 1, self.image_patch, self.image_patch]), np.array(self.y).reshape([len(self.y), 1, 64]), state, reward)
    
    def render(self, action):

        self.screen.fill((255, 255, 255)) 
        
        for row in range(self.num_rows):
            for col in range(self.num_columns):
                cell_left = col * self.cell_size
                cell_top = row * self.cell_size
                
                pygame.draw.rect(self.screen, (0, 0, self.color[row, col]), (cell_left, cell_top, self.cell_size, self.cell_size))
                
                if self.seen[row, col] == 1:
                    pygame.draw.rect(self.screen, (0, self.color[row, col], 0), (cell_left, cell_top, self.cell_size, self.cell_size))
                
                if [row, col] in self.pos_X:
                    pygame.draw.rect(self.screen, (self.color[row, col], 0, 0), (cell_left, cell_top, self.cell_size, self.cell_size))
                
                    
                

        pygame.display.update()

            
    def reset(self, start = [50, 50]):
        self.pos_X = []
        self.X = []
        self.y = []
        self.seen = np.zeros([100, 100])
        self.X.append(self.image[start[0]-int((self.image_patch-1)/2): start[0] + int((self.image_patch+1)/2), start[1]-int((self.image_patch-1)/2): start[1] + int((self.image_patch+1)/2)])
        self.y.append(spectra[start[0], start[1]])
        self.seen = np.zeros([100, 100])
        self.pos = start
        self.pos_X = [start]
        
        for i in range(self.radius):
                for j in range(self.radius):
                    self.seen[start[0]+i, start[1]+j] = 1
                    self.seen[start[0]-i, start[1]+j] = 1
                    self.seen[start[0]+i, start[1]-j] = 1
                    self.seen[start[0]-i, start[1]-j] = 1
        
        
        self.observation_space = spaces.Tuple((spaces.Discrete(self.num_rows), spaces.Discrete(self.num_columns)))
        
        pygame.init()
        self.cell_size = 8
        
        self.screen = pygame.display.set_mode((self.num_columns * self.cell_size, self.num_rows * self.cell_size))      
    
    

In [6]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)


    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():

            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[random.randint(0, n_actions-1)]], device=device, dtype=torch.long)
    
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()

    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 10000
TAU = 0.005
LR = 1e-4

start = [50, 50]
image_patch = 5
image, spectra = pola[:,:,0], pola[:,:,::2][:,:,0:64]
radius = int((image_patch - 1)/2)

env = environment(image, spectra, start = start, image_patch = image_patch)

n_actions = (100 - 2*radius)**2
n_observations = 2 + image_patch**2 + 64

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())



optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)

model = im2spec((image_patch, image_patch), 64, 10)

memory = ReplayMemory(10000)


steps_done = 0

pos_X, X, y, state, reward = env.state(model)
model = trainer(model, X, y, X, y, num_epochs=1, savename="im2spec_lv{}".format(10)).run()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
env.render(start)
for i in range(100):
    action = select_action(state)
    env.step([action.item()//(100 - 2*radius)+radius, action.item()%(100 - 2*radius)+radius])
    
    pos_X, X, y, observation, reward = env.state(model)
        
    model = trainer(model, X, y, X, y, num_epochs=1, savename="im2spec_lv{}".format(10)).run()
    reward = torch.tensor([reward], device=device)


    next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

    # Store the transition in memory
    memory.push(state, action, next_state, reward)

    # Move to the next state
    state = next_state

    # Perform one step of the optimization (on the policy network)
    optimize_model()

    # Soft update of the target network's weights
    # θ′ ← τ θ + (1 −τ )θ′
    target_net_state_dict = target_net.state_dict()
    policy_net_state_dict = policy_net.state_dict()
    for key in policy_net_state_dict:
        target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
    target_net.load_state_dict(target_net_state_dict)
    env.render([action.item()//(100 - 2*radius)+radius, action.item()%(100 - 2*radius)+radius])
    pygame.time.wait(100)

Epoch: 1... Training loss: 282.53265... Test loss: 279.21411
Epoch: 1... Training loss: 256.07867... Test loss: 255.37231
Epoch: 1... Training loss: 132.18156... Test loss: 133.96418
Epoch: 1... Training loss: 74.02675... Test loss: 76.1176
Epoch: 1... Training loss: 77.11907... Test loss: 80.4603
Epoch: 1... Training loss: 55.25793... Test loss: 57.08502
Epoch: 1... Training loss: 60.65109... Test loss: 63.49926
Epoch: 1... Training loss: 49.31269... Test loss: 51.03321
Epoch: 1... Training loss: 48.6919... Test loss: 50.99257
Epoch: 1... Training loss: 47.02327... Test loss: 48.69957
Epoch: 1... Training loss: 43.14386... Test loss: 44.77903
Epoch: 1... Training loss: 36.51435... Test loss: 37.62475
Epoch: 1... Training loss: 34.88717... Test loss: 35.95741
Epoch: 1... Training loss: 31.1224... Test loss: 32.27818
Epoch: 1... Training loss: 30.40655... Test loss: 31.13335
Epoch: 1... Training loss: 28.95173... Test loss: 30.06375
Epoch: 1... Training loss: 29.76359... Test loss: 30.1